# Εξαγωγή Μεταδεδομένων Θερμικών Εικόνων
## Χρήση του εργαλείου [exiftool](https://exiftool.org/)
### Παραγωγή csv αρχείων με τα μεταδεδομένα των ετικετών εντός των jpeg 

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

### Μεταφορά του συστήματος στο directory που βρίσκεται το πρόγραμμα `exiftool.exe`

In [8]:
cd ..

C:\Users\plouk\Adiposer


### H σύνταξη των εντολών για τη χρήση του εργαλείου μεταδεδομένων

In [ ]:
source = module_path + '/data/raw/0h/mouse_1/'
dest = module_path + '/data/interim
#!exiftool.exe -csv {source} > {a}.csv
a = !exiftool.exe -csv {source} > ola.csv

### Ορισμοί των Path για το dataset

In [2]:
PROJECT_DIR = module_path
RAW_DIR = PROJECT_DIR + "/data/raw"
INTERIM_DIR = PROJECT_DIR + "/data/interim"
mode = 0o755
sample_hours = ['0h', '24h', '48h', '72h', '96h', '120h', '144h', '192h', '240h']
mouse_ids = ['mouse_1', 'mouse_2', 'mouse_3', 'mouse_4', 'mouse_5']

### Κατασκευή directory interim και subdirectories για τα μεταδεδομένα
#### Mορφή 
`data/interim`
- 0h
    - mouse_1.csv
    - mouse_2.csv ...
- 24h
    - mouse_1.csv
    - mouse_2.csv ...
    

In [4]:
# 1. Αρχικοποίηση INTERIM_DIR με τα sample_hours directories
# print('******* 1. Initialized /data/interim/[sample_hours] directories **************')
[os.mkdir(INTERIM_DIR + f'/{sample_hour}', mode) for sample_hour in sample_hours]
# 2. Κατασκευή των subdirectories με τα mouse_ids
# print('******* 2. Creating /data/interim/[sample_hours]/[mouse_ids] directories **************')
# [os.mkdir(INTERIM_DIR + f'/{sample_hour}/{mouse_id}', mode) for sample_hour in sample_hours for mouse_id in mouse_ids]

[None, None, None, None, None, None, None, None, None]

In [7]:
processed_files = 0
for i, (raw_root, raw_dirs, raw_files) in enumerate(os.walk(RAW_DIR)):
    # raw_root : string of a POSIX-style directory
    # raw_dirs : list of subdirectories basenames
    # raw_files: list of filenames in the directory
    print(f'Iteration #{i}')
    print(f'  Basename Root dir: {os.path.basename(raw_root)}')
    print(f'  Found DIRs: {str(raw_dirs)}')

    if len(raw_dirs) == 0: # 3. Οχι άλλα subdirectories - Δεδομένα εικόνων 
        print(f'******* {len(raw_files)} Files Found at {raw_root}')

        parent_dir = os.path.abspath(os.path.join(raw_root, os.pardir))
        sample_h = os.path.basename(parent_dir)

        if raw_root.endswith('1'):
            mouse_id = 'mouse_1'
        elif raw_root.endswith('2'):
            mouse_id = 'mouse_2'
        elif raw_root.endswith('3'):
            mouse_id = 'mouse_3'
        elif raw_root.endswith('4'):
            mouse_id = 'mouse_4'
        elif raw_root.endswith('5'):
            mouse_id = 'mouse_5'

        SAVE_DIR = INTERIM_DIR + f'{sample_h}/{mouse_id}'
        FILE_NAME = SAVE_DIR + f'/{mouse_id}.csv'
        
        !exiftool.exe -csv {raw_root} > {FILE_NAME}.csv

        #print(f'Base directory: {os.path.basename(raw_root)}')
        print(f'Base directory: {raw_root}')
        print(f'Saving at: {SAVE_DIR}')
        # full_path_target = [raw_root + '/' + raw_file for raw_file in raw_files]
        # processed_files += process_targets(full_path_target, raw_files, SAVE_DIR)

        print(f'------------ Total Processed Files {processed_files} ------------------')

Iteration #0
  Basename Root dir: raw
  Found DIRs: ['0h', '120h', '144h', '192h', '240h', '24h', '48h', '72h', '96h']
Iteration #1
  Basename Root dir: 0h
  Found DIRs: ['mouse_1', 'mouse_2', 'mouse_3', 'mouse_4', 'mouse_5']
Iteration #2
  Basename Root dir: mouse_1
  Found DIRs: []
******* 109 Files Found at C:\Users\plouk\Adiposer/data/raw\0h\mouse_1
Base directory: C:\Users\plouk\Adiposer/data/raw\0h\mouse_1
Saving at: C:\Users\plouk\Adiposer/data/interim0h/mouse_1
------------ Total Processed Files 0 ------------------
Iteration #3
  Basename Root dir: mouse_2
  Found DIRs: []
******* 67 Files Found at C:\Users\plouk\Adiposer/data/raw\0h\mouse_2


The system cannot find the path specified.


Base directory: C:\Users\plouk\Adiposer/data/raw\0h\mouse_2
Saving at: C:\Users\plouk\Adiposer/data/interim0h/mouse_2
------------ Total Processed Files 0 ------------------
Iteration #4
  Basename Root dir: mouse_3
  Found DIRs: []
******* 58 Files Found at C:\Users\plouk\Adiposer/data/raw\0h\mouse_3


The system cannot find the path specified.


Base directory: C:\Users\plouk\Adiposer/data/raw\0h\mouse_3
Saving at: C:\Users\plouk\Adiposer/data/interim0h/mouse_3
------------ Total Processed Files 0 ------------------
Iteration #5
  Basename Root dir: mouse_4
  Found DIRs: []
******* 52 Files Found at C:\Users\plouk\Adiposer/data/raw\0h\mouse_4


...

The system cannot find the path specified.


Base directory: C:\Users\plouk\Adiposer/data/raw\96h\mouse_4
Saving at: C:\Users\plouk\Adiposer/data/interim96h/mouse_4
------------ Total Processed Files 0 ------------------
Iteration #54
  Basename Root dir: mouse_5
  Found DIRs: []
******* 82 Files Found at C:\Users\plouk\Adiposer/data/raw\96h\mouse_5


The system cannot find the path specified.


Base directory: C:\Users\plouk\Adiposer/data/raw\96h\mouse_5
Saving at: C:\Users\plouk\Adiposer/data/interim96h/mouse_5
------------ Total Processed Files 0 ------------------


The system cannot find the path specified.
